In [75]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../input/credit-card-scoring/data/Training.csv', na_values=np.nan, encoding_errors='replace')

In [3]:
df.head()

In [4]:
df.info()

In [5]:
df = df.drop('EDUCATION', axis = 1)
df = df.dropna()

In [6]:
df.info()

In [48]:
to_train = df[['AGE', 'QUANT_DEPENDANTS', 'RESIDENCE_TYPE', 'MONTHS_IN_RESIDENCE', 'FLAG_RESIDENCE_TOWN=WORKING_TOWN', \
              'FLAG_RESIDENCE_STATE=WORKING_STATE', 'MONTHS_IN_THE_JOB', 'PROFESSION_CODE', 'MATE_INCOME', \
               'FLAG_RESIDENCIAL_ADDRESS=POSTAL_ADDRESS', 'FLAG_OTHER_CARD', 'QUANT_BANKING_ACCOUNTS', \
              'FLAG_MOBILE_PHONE', 'FLAG_CONTACT_PHONE', 'PERSONAL_NET_INCOME']]

In [49]:
target = df['TARGET_LABEL_BAD=1']

In [56]:
to_train.dtypes

In [83]:
from sklearn.preprocessing import FunctionTransformer, MinMaxScaler, OneHotEncoder
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.ensemble import GradientBoostingClassifier

text_columns = [i for i in to_train.columns if df[i].dtype == 'object']
numeric_columns = [i for i in to_train.columns if (df[i].dtype == 'float64' or df[i].dtype == 'int64')]

get_numeric_data = FunctionTransformer(lambda x: x[numeric_columns], validate=False)
get_text_data = FunctionTransformer(lambda x: x[text_columns], validate=False)

scaler = MinMaxScaler()
ohe = OneHotEncoder()

steps_numeric = [('get_data', get_numeric_data), ('processing', scaler)]
steps_text = [('get_data', get_text_data), ('processing', ohe)]

pipe_numeric = Pipeline(steps_numeric)
pipe_text = Pipeline(steps_text)
estimator = GradientBoostingClassifier()

features = FeatureUnion([('nums', pipe_numeric), ('text', pipe_text)])
model = Pipeline([('input', features), ('booster', estimator)])

In [84]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(to_train, target, random_state=42)

In [87]:
model.fit(X_train, y_train)

In [93]:
from sklearn.metrics import f1_score

validation = model.predict(X_valid)
results_val = f1_score(y_valid, validation)

print(results_val)

In [91]:
model.score(X_valid, y_valid)